# CareerCon 2019 - Help Navigate Robots

Robots are smart… by design. To fully understand and properly navigate a task, however, they need input about their environment.

**Objective:** Improve the navigation of robots without assistance across many different surfaces, so they won’t fall down on the job.

**Study:** In this notebook I do an analysis with the data to show that there are different patterns for different surfaces, I also purpose a new categorization based in how easy is to move in a given surface.

### Content:
1. [Information about the surface](#Information-about-the-surface);
2. [Surface distribution](#Surface-distribution);
3. [Correlation between surface and speed](#Correlation-between-surface-and-speed);
4. [New surface categories](#New-surface-categories)
5. [Results](#Results)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### Visualization ###
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

### Utils ###
import warnings
warnings.filterwarnings("ignore")

In [2]:
movement_cols = ['orientation_X', 'orientation_Y', 'orientation_Z',
                 'orientation_W', 'angular_velocity_X', 'angular_velocity_Y',
                 'angular_velocity_Z', 'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z']
label = ['surface']

train = pd.read_csv('../input/carrercon-join-of-x-and-y-train/x_y_train/x_y_train.csv')

## Information about the surface

**Carpet:** It's really bad.
![image](https://images-na.ssl-images-amazon.com/images/I/61%2B2hk3eDRL.jpg)

**Concrete:** Best floor.
![image](https://www.floormastervt.com/wp-content/uploads/2016/05/polish1.jpg?x40097)

**Fine concrete:** Good superficies to work with.
![image](https://flowingstone.co.nz/wp-content/uploads/2016/08/Concrete-Types-1.jpeg)

**Hard tiles:** Can be difficult depending of how the tittle can be organized.
* Good: ![image](https://previews.123rf.com/images/mehaniq/mehaniq1704/mehaniq170400385/75713143-the-texture-of-solid-granite-tiles-hard-and-slippery-gray-granite-surface-with-a-relief-discharge-wh.jpg)
* Bad: ![image](https://www.cmmonline.com/images/content/magazines/2015/06-2015/06-2015-tips-detail.jpg?Action=thumbnail&Width=750&Height=392&algorithm=fill_proportional)

**Hard tiles large:** Fits[](http://) in the *bad* case for surface hard tile.

**Soft PVC:** Better than soft tiles but depending of the weight of the robot.
![image](https://sc01.alicdn.com/kf/HTB1N.KJKpXXXXbfXFXXq6xXFXXX9/Soft-PVC-kindergarten-flooring-in-bright-color.jpg_350x350.jpg)

**Soft tiles:** Not as bad as the carpet but it's lower and can have irregularities.
![image](https://assets.incstores.com/products/reviews/5-8%20Soft%20Tiles-6.jpg)

**Tiled:** Bad since there will be many space.
![image](https://us.123rf.com/450wm/ericmilos/ericmilos1102/ericmilos110200013/8713937-tiled-concrete-pavement.jpg?ver=6)

**Wood:** Very good surface, except in cases that the join is uneven.
![image](https://static.flooringsupplies.co.uk/images/grain/CAS1472.jpg)

## Surface distribution

* Most surfaces are concrete, what is a good surface for the robot to move.

In [3]:
surface_distribution = (train
                       .groupby('surface')
                       .size()
                       .to_frame('count')
                       .reset_index()
                       .sort_values('count', ascending=True))

data = [go.Bar(
            y=surface_distribution['surface'],
            x=surface_distribution['count'],
            orientation = 'h',
            text=surface_distribution['count']
    )]

layout = go.Layout(
    title='Distribution of surfaces',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## Correlation between surface and speed
In order to prove the hypothesis that the floor is correlated of how well will the robot move.

In [4]:
metrics = (train[movement_cols+label]
                   .groupby('surface')
                   .agg(['min','max','mean', 'std'])
              )

In [5]:
surfaces = list(surface_distribution.sort_values('surface').surface)
surfaces

['carpet',
 'concrete',
 'fine_concrete',
 'hard_tiles',
 'hard_tiles_large_space',
 'soft_pvc',
 'soft_tiles',
 'tiled',
 'wood']

In [6]:
metrics

orientation_X             ...    linear_acceleration_Z          
                                 min      max    ...                     mean       std
surface                                          ...                                   
carpet                      -0.98859  0.38137    ...                -9.368058  2.288149
concrete                    -0.98862  0.98889    ...                -9.356408  3.726617
fine_concrete               -0.89089  0.98642    ...                -9.356070  2.047193
hard_tiles                   0.46074  0.74929    ...                -9.332681  0.599659
hard_tiles_large_space      -0.71222  0.98872    ...                -9.365567  4.866550
soft_pvc                    -0.97853  0.98910    ...                -9.386315  1.235721
soft_tiles                  -0.96457  0.98874    ...                -9.359474  0.485342
tiled                       -0.98910  0.95521    ...                -9.357045  3.489906
wood                        -0.98872  0.98865    ...                -9.364265  2.134775

[9 rows x 40 columns]

In [7]:
metrics.columns

MultiIndex(levels=[['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z'], ['min', 'max', 'mean', 'std']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

In [8]:
def bars(df, col_x, col_y):
    return go.Bar(
            y=df[col_y],
            x=df[col_x],
            orientation = 'h',
            text=df[col_x],
            name=col_x
    )

In [9]:
metrics_name = ['std', 'mean', 'max', 'min']

In [10]:
orientation_x = metrics.orientation_X
orientation_x['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(orientation_x, m, 'surface'))

layout = go.Layout(
    title='Orientation X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [11]:
orientation_y = metrics.orientation_Y
orientation_y['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(orientation_y, m, 'surface'))

layout = go.Layout(
    title='Orientation Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [12]:
orientation_Z = metrics.orientation_Z
orientation_Z['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(orientation_Z, m, 'surface'))

layout = go.Layout(
    title='Orientation Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [13]:
orientation_W = metrics.orientation_W
orientation_W['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(orientation_W, m, 'surface'))

layout = go.Layout(
    title='Orientation W',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [14]:
angular_velocity_X = metrics.angular_velocity_X
angular_velocity_X['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_X, m, 'surface'))

layout = go.Layout(
    title='Angular velocity X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [15]:
angular_velocity_Y = metrics.angular_velocity_Y
angular_velocity_Y['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_Y, m, 'surface'))

layout = go.Layout(
    title='Angular velocity Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [16]:
angular_velocity_Z = metrics.angular_velocity_Z
angular_velocity_Z['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_Z, m, 'surface'))

layout = go.Layout(
    title='Angular velocity Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [17]:
linear_acceleration_X = metrics.linear_acceleration_X
linear_acceleration_X['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_X, m, 'surface'))

layout = go.Layout(
    title='Linear Acceleration X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [18]:
linear_acceleration_Y = metrics.linear_acceleration_Y
linear_acceleration_Y['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_Y, m, 'surface'))

layout = go.Layout(
    title='Linear Acceleration Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [19]:
linear_acceleration_Z = metrics.linear_acceleration_Z
linear_acceleration_Z['surface'] = surfaces

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_Z, m, 'surface'))

layout = go.Layout(
    title='Linear Acceleration Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

The charts above prove that the metrics are different for each kind of surface.

## New surface categories

Now let's try and creat new categories based in how hard is to move in each floor.

In [20]:
new_cat = {'carpet': 'bad', 'concrete': 'good', 'fine_concrete': 'good',
           'hard_tiles': 'good', 'hard_tiles_large_space': 'bad','soft_pvc': 'medium',
           'soft_tiles': 'medium', 'tiled': 'bad', 'wood': 'good'}

In [21]:
train['type_surface'] = train['surface'].map(new_cat)

In [22]:
train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface,type_surface
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,13,fine_concrete,good
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003386,0.33995,1.5064,-9.4128,13,fine_concrete,good
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,13,fine_concrete,good
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960,13,fine_concrete,good
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410,13,fine_concrete,good


In [23]:
metrics_new_cat = (train[movement_cols+['type_surface']]
                   .groupby('type_surface')
                   .agg(['min','max','mean', 'std'])
              )

In [24]:
type_surface = ['bad', 'good', 'medium']

In [25]:
metrics_new_cat

orientation_X             ...    linear_acceleration_Z          
                       min      max    ...                     mean       std
type_surface                           ...                                   
bad               -0.98910  0.98872    ...                -9.361700  3.792861
good              -0.98872  0.98889    ...                -9.358751  2.922112
medium            -0.97853  0.98910    ...                -9.378568  1.074429

[3 rows x 40 columns]

In [26]:
orientation_x = metrics_new_cat.orientation_X
orientation_x['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(orientation_x, m, 'type_surface'))

layout = go.Layout(
    title='Orientation X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [27]:
orientation_Y = metrics_new_cat.orientation_Y
orientation_Y['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(orientation_Y, m, 'type_surface'))

layout = go.Layout(
    title='Orientation Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [28]:
orientation_Z = metrics_new_cat.orientation_Z
orientation_Z['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(orientation_Z, m, 'type_surface'))

layout = go.Layout(
    title='Orientation Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [29]:
orientation_W = metrics_new_cat.orientation_W
orientation_W['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(orientation_W, m, 'type_surface'))

layout = go.Layout(
    title='Orientation W',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [30]:
angular_velocity_X = metrics_new_cat.angular_velocity_X
angular_velocity_X['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_X, m, 'type_surface'))

layout = go.Layout(
    title='Angular velocity X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [31]:
angular_velocity_Y = metrics_new_cat.angular_velocity_Y
angular_velocity_Y['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_Y, m, 'type_surface'))

layout = go.Layout(
    title='Angular velocity Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [32]:
angular_velocity_Z = metrics_new_cat.angular_velocity_Z
angular_velocity_Z['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(angular_velocity_Z, m, 'type_surface'))

layout = go.Layout(
    title='Angular velocity Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [33]:
linear_acceleration_X = metrics_new_cat.linear_acceleration_X
linear_acceleration_X['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_X, m, 'type_surface'))

layout = go.Layout(
    title='Linear Acceleration X',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [34]:
linear_acceleration_Y = metrics_new_cat.linear_acceleration_Y
linear_acceleration_Y['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_Y, m, 'type_surface'))

layout = go.Layout(
    title='Linear Acceleration Y',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [35]:
linear_acceleration_Z = metrics_new_cat.linear_acceleration_Y
linear_acceleration_Z['type_surface'] = type_surface

data = []

for m in metrics_name:
    data.append(bars(linear_acceleration_Z, m, 'type_surface'))

layout = go.Layout(
    title='Linear Acceleration Z',
    titlefont=dict(size=20),
    width=800,
    height=600
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

There's a significant difference for the mean value of *good* surfaces in : orientation_Y, orientation_Z, angular_velocity_X, angular_velocity_Y, 
angular_velocity_Z, linear_acceleration_X.


## Results

As expected, it was provode that the kind of surface impacts the movement metrics. Besides that, those surfaces can be futher classified in *good*, *medium* and *bad*.